### setup dataset

In [1]:
import torch
from torch.utils.data import Dataset
import pandas as pd

class SiblingsVideoDataset(Dataset):
    def __init__(self, path_to_labels, num_of_cls, image_processor):

        # TODO fix this
        label_1 = pd.read_csv(path_to_labels, header=1).iloc[:, :num_of_cls+1]
        label_2 = pd.read_csv(path_to_labels, header=1 ).iloc[:, :num_of_cls+1]
        # label1_3 = pd.read_csv('label_3/回应情况-表格 1.csv')

        label = pd.concat([label_1, label_2], axis=0)
        label = label1.rename(columns={"Unnamed: 0":"切片ID"})

        self.video_paths = video_paths
        self.labels = labels
        self.image_processor = image_processor

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        label = self.labels[idx]
        video_frames = self.image_processor.read_video(video_path)
        inputs = self.image_processor(video_frames, return_tensors="pt")
        return inputs, torch.tensor(label)

/home/ansonlo/anaconda3/envs/torch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using some sota video classification model in transformers  
[VideoMAE](https://huggingface.co/docs/transformers/main/en/model_doc/videomae#transformers.VideoMAEForVideoClassification)   
[notebook](https://github.com/huggingface/notebooks/blob/main/examples/video_classification.ipynb)  

In [5]:
from transformers import AutoFeatureExtractor, AutoModelForVideoClassification

extractor = AutoFeatureExtractor.from_pretrained("meermoazzam41/videomae-base-finetuned-human-activity-classification")

model = AutoModelForVideoClassification.from_pretrained("meermoazzam41/videomae-base-finetuned-human-activity-classification")

/home/ansonlo/anaconda3/envs/torch/lib/python3.8/site-packages/transformers/models/videomae/feature_extraction_videomae.py:28: FutureWarning: The class VideoMAEFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use VideoMAEImageProcessor instead.
  warnings.warn(


In [4]:
from transformers import VideoMAEImageProcessor, VideoMAEForVideoClassification, VideoMAEConfig


model_ckpt = "MCG-NJU/videomae-base"
config = VideoMAEConfig.from_pretrained(model_ckpt)

image_processor = VideoMAEImageProcessor.from_pretrained(model_ckpt)
model = VideoMAEForVideoClassification.from_pretrained(model_ckpt)

Some weights of the model checkpoint at MCG-NJU/videomae-base were not used when initializing VideoMAEForVideoClassification: ['decoder.decoder_layers.3.output.dense.weight', 'decoder.decoder_layers.3.layernorm_after.bias', 'decoder.decoder_layers.1.attention.attention.q_bias', 'decoder.decoder_layers.0.attention.attention.key.weight', 'decoder.decoder_layers.2.intermediate.dense.weight', 'decoder.decoder_layers.3.intermediate.dense.weight', 'decoder.decoder_layers.2.attention.output.dense.bias', 'decoder.decoder_layers.1.attention.attention.query.weight', 'decoder.decoder_layers.3.output.dense.bias', 'decoder.decoder_layers.0.intermediate.dense.weight', 'decoder.decoder_layers.0.layernorm_after.weight', 'decoder.decoder_layers.2.layernorm_before.weight', 'decoder.decoder_layers.2.attention.output.dense.weight', 'decoder.decoder_layers.0.attention.output.dense.weight', 'decoder.decoder_layers.2.output.dense.weight', 'mask_token', 'decoder.decoder_layers.0.output.dense.bias', 'decoder.d

In [ ]:
video_path = "path/to/your/video.mp4"
video_frames = image_processor.read_video(video_path)

inputs = image_processor(video_frames, return_tensors="pt")


with torch.no_grad():
    logits = model(**inputs).logits


predicted_class = logits.argmax(dim=-1).item()